# Linear Regression Modelling with Elastic Net

Use 'quick & dirty' custom functions from my LinRegModel class to find an optimized baseline model.
Build an evaluation pipeline to evaluate different treatments for the training data (using that baseline model).
Save final pipeline / model for later feature importance evaluation (in notebook 4).

*Note: An issue that was not completely / elegantly solved: If I use sklearns ElasticNet() class with integrated gridsearch as regressor step inside the modelling pipeline, I don't know how get the best params out of it. So I could make a preprocessing pipeline first (--> use fit\_transform() to return a transformed data set, see Appendix) and then use the ElasticNet() class separately on that data. But then OHE would have to be made outside the pipe, because train and test sets get different dimensions using this approach (and if I would transform the whole dataset at once I would have data leakage). Or I could evaluate Elastic Net params with regular grid search. But was to lazy in the end to do that properly ...*

**Data Sources**

- `data/raw/train.csv`: Training set from [kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).

**Output**

- `data/interim/train_opti_EN.csv`: Best feature set configuration for pipeline.
- `models/full_pipe_final.pkl`: Best pipeline / model.



**Changes**

- 2019-03-22: Start notebook
- 2019-03-29: Finish notebook, save best model


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries,-load-data" data-toc-modified-id="Import-libraries,-load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries, load data</a></span></li><li><span><a href="#Go-quick-&amp;-dirty" data-toc-modified-id="Go-quick-&amp;-dirty-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Go quick &amp; dirty</a></span></li><li><span><a href="#General-data-pre-processing-(outside-of-sklearn-pipeline)" data-toc-modified-id="General-data-pre-processing-(outside-of-sklearn-pipeline)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>General data pre-processing (outside of sklearn pipeline)</a></span><ul class="toc-item"><li><span><a href="#General-pre-processing" data-toc-modified-id="General-pre-processing-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>General pre-processing</a></span></li></ul></li><li><span><a href="#Explore-different-feature-set-options" data-toc-modified-id="Explore-different-feature-set-options-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Explore different feature set options</a></span><ul class="toc-item"><li><span><a href="#Define-pipeline-with-CV-to-evaluate-different-options" data-toc-modified-id="Define-pipeline-with-CV-to-evaluate-different-options-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Define pipeline with CV to evaluate different options</a></span></li><li><span><a href="#Explore-feature-sets" data-toc-modified-id="Explore-feature-sets-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Explore feature sets</a></span><ul class="toc-item"><li><span><a href="#Standard-train-set-(with-NaN-and-all-Outliers)" data-toc-modified-id="Standard-train-set-(with-NaN-and-all-Outliers)-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Standard train set (with NaN and all Outliers)</a></span></li><li><span><a href="#Train-set-without-columns-containing-NaN" data-toc-modified-id="Train-set-without-columns-containing-NaN-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Train set without columns containing NaN</a></span></li><li><span><a href="#Full-train-set-with-outliers-removed-for-top-correlating-columns" data-toc-modified-id="Full-train-set-with-outliers-removed-for-top-correlating-columns-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Full train set with outliers removed for top correlating columns</a></span></li><li><span><a href="#Full-train-set-with-outliers-removed-and-multi-correlation-columns-removed" data-toc-modified-id="Full-train-set-with-outliers-removed-and-multi-correlation-columns-removed-4.2.4"><span class="toc-item-num">4.2.4&nbsp;&nbsp;</span>Full train set with outliers removed and multi-correlation columns removed</a></span></li></ul></li><li><span><a href="#Final-Tuning-&amp;-Evaluation" data-toc-modified-id="Final-Tuning-&amp;-Evaluation-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Final Tuning &amp; Evaluation</a></span></li></ul></li><li><span><a href="#Apppendix:-Experiment-with-preprocessing-pipe-only" data-toc-modified-id="Apppendix:-Experiment-with-preprocessing-pipe-only-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Apppendix: Experiment with preprocessing pipe only</a></span></li></ul></div>

---

## Import libraries, load data

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from tqdm import tqdm

from scipy import stats
from scipy.stats import norm, skew

from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# My functions
import EDA_functions as EDA
import cleaning_functions as cleaning
from linRegModel_class import LinRegModel

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns #, sns.set_style('whitegrid')
color = 'rebeccapurple'
%matplotlib inline

# Display settings
from IPython.display import display
pd.options.display.max_columns = 100

In [2]:
# Load data
raw_data = pd.read_csv('data/raw/train.csv')

# Check shape
display(raw_data.shape)

(1460, 81)

In [3]:
# Load variables from notebook 1
%store -r cols_to_del
%store -r cols_to_log
%store -r outliers_to_del
%store -r top_corr_columns

## Go quick & dirty
Use my 'quick & dirty' function for a baseline model on unprocessed data.

In [4]:
# Initialize a scikit-learn model object of choice
model_simple = ElasticNetCV(alphas=[0.03, 0.05, 0.09], copy_X=True, cv=5, eps=0.001, 
                            fit_intercept=True, l1_ratio=[0.6, 0.9, 1.0], max_iter=3000, 
                            n_alphas=100, n_jobs=-1)

# Create an instance of the LinRegModel class by passing df, target variable and model object
elastic_net_simple = LinRegModel(raw_data, 'SalePrice', model_simple)

# Output instance
display(elastic_net_simple)

ElasticNetCV(alphas=[0.03, 0.05, 0.09], copy_X=True, cv=5, eps=0.001,
       fit_intercept=True, l1_ratio=[0.6, 0.9, 1.0], max_iter=3000,
       n_alphas=100, n_jobs=-1, normalize=False, positive=False,
       precompute='auto', random_state=None, selection='cyclic',
       tol=0.0001, verbose=0)

In [5]:
# Perform the modelling
elastic_net_simple.go_quickDirty()

C:\Users\r2d4\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\r2d4\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\r2d4\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\r2d4\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase t

C:\Users\r2d4\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\r2d4\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\r2d4\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\r2d4\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase t

In [6]:
# Output result
elastic_net_simple

ElasticNetCV(alphas=[0.03, 0.05, 0.09], copy_X=True, cv=5, eps=0.001,
       fit_intercept=True, l1_ratio=[0.6, 0.9, 1.0], max_iter=3000,
       n_alphas=100, n_jobs=-1, normalize=False, positive=False,
       precompute='auto', random_state=None, selection='cyclic',
       tol=0.0001, verbose=0)

RMSE on test data 33373.76, r2-score 0.80.

In [7]:
# Check best values
print(model_simple.alpha_)
print(model_simple.l1_ratio_)
print(model_simple.n_iter_)

0.09
0.9
3000


## General data pre-processing (outside of sklearn pipeline)
Pre-processing steps that take place before data is pipelined

### General pre-processing

In [8]:
# Disable warning
pd.set_option('mode.chained_assignment', None)

# Create and clean training set with variables from the EDA notebook
train_data = (raw_data
              .pipe(cleaning.change_dtypes, cols_to_category=raw_data.select_dtypes(object))
              .pipe(cleaning.delete_columns, cols_to_delete=cols_to_del)
              .pipe(cleaning.apply_log, cols_to_transform=cols_to_log)
             )

train_data.drop(outliers_to_del, inplace=True)
train_data.dropna(subset=['MasVnrArea', 'MasVnrType', 'Electrical'], inplace=True);

'MiscFeature successfully deleted'

'PoolQC successfully deleted'

'FireplaceQu successfully deleted'

'Alley successfully deleted'

'Id successfully deleted'

'Fence successfully deleted'

In [9]:
# check results
display(train_data.shape)

(1447, 75)

## Explore different feature set options

### Define pipeline with CV to evaluate different options

In [10]:
def evaluate_feature_sets(df, reg, scorer, cv=StratifiedKFold(3)):
    """Build a pipeline for evaluating different combinations of data, models
    and scorers with stratified crossevaluation. The pipeline performs 
    necessary transformations onf categorical and numeric features and 
    evaluates the imputation strategy or if numerical data is to scale or
    not. In this notebook my only changing variable will be the input data
    
    ARGUMENTS:
        df: dataframe, input data for modelling
        reg: sklearn model instance, a baseline model
        scorer: string or make_score() object (?) 
        
    RETURNS:
        grid_results: dict, best parameters for model
        best_score: float, highest score value - watch out if you have a loss
            function. Then you have to search for the minimal score value
    """
    
    # Split input features and target label
    X_train = df.drop('SalePrice', axis=1)
    y_train = df['SalePrice'].copy()
    
    # Define cat and num feature columns
    categorical_features = X_train.select_dtypes(include=['category']).columns
    numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
    assert len(categorical_features) + len(numeric_features) == df.shape[1] - 1
    
    ## Assemble pipeline (define function)
    
    # level 1 - two separate pipes for cat and num features
    numeric_transformer = Pipeline(steps=[
        ('imputer_n', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
            ])

    categorical_transformer = Pipeline(steps=[
        ('imputer_c', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ohe', OneHotEncoder(handle_unknown='ignore')),
            ])

    # level 2 - wrap the two level 1 pipes into a ColumnTransformer
    preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features),
                         ])

    # level 3 - pipe it with a classifier
    full_pipe = Pipeline(steps=[
                       ('preprocessor', preprocessor),
                       ('reg', model_simple),
                               ]) 
    
    # Evaluate imputing strategy for missing num values and scaling
    parameters = {
        'preprocessor__num__imputer_n__strategy': ['mean', 'median'],
        'preprocessor__num__scaler' : [None, StandardScaler()]
                 }

    cv = GridSearchCV(full_pipe, param_grid=parameters, scoring=scorer, n_jobs=-1, iid=False,
                      cv=cv, error_score='raise', return_train_score=False, verbose=1)

    grid = cv.fit(X_train, y_train) 
    grid_results = grid.cv_results_

    # Here I have to go for the smallest score (CV expects utility function
    # and not cost function, see Hands-OnML p. 70)
    best_score = np.sqrt(np.min(grid_results['mean_test_score']))
    
    return grid_results, best_score

In [11]:
# Define input parameters
scorer = make_scorer(mean_squared_error)
reg = elastic_net_simple # 'optimized baseline model'
cv = 3

### Explore feature sets

#### Standard train set (with NaN and all Outliers)

In [12]:
# Run pipeline
grid_results, best_score = evaluate_feature_sets(train_data, reg=reg, scorer=scorer, cv=cv)

# Print best score
print(best_score)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    9.4s finished


0.13817449507839052


In [13]:
display(pd.DataFrame(grid_results).nsmallest(1, 'mean_test_score'))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessor__num__imputer_n__strategy,param_preprocessor__num__scaler,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
1,1.362899,0.231008,0.080529,0.013988,mean,"StandardScaler(copy=True, with_mean=True, with...",{'preprocessor__num__imputer_n__strategy': 'me...,0.018117,0.020913,0.018247,0.019092,0.001289,4


**Result:** Best score for imputation with mean and applied StandardScaler(). (The latter has an impact but imputation with 'mean' or 'median' leads to more or less the same result.)

#### Train set without columns containing NaN

In [14]:
# Create List of Columns containing NaN
nan_cols = []
for col in train_data.columns:
    if train_data[col].isnull().sum() > 0:
        nan_cols.append(col)

In [15]:
# Check results
nan_cols

['LotFrontage',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [16]:
# Create train set without missing values (drop nan_cols)
train_data_reduced = train_data.drop(nan_cols, axis=1)

assert train_data_reduced.isnull().sum().sum() == 0
assert train_data_reduced.shape[1] == train_data.shape[1] - len(nan_cols)

In [17]:
# Run pipeline
grid_results, best_score = evaluate_feature_sets(train_data_reduced, 
                                                 reg=reg, scorer=scorer, cv=cv)

# Print best score
print(best_score)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    3.7s finished


0.13817450160297168


**Result:** Results are really, really close. Imputing with Mean scores slightly better than elimination of the columns.

#### Full train set with outliers removed for top correlating columns

In [18]:
# Remove Outliers for remaining top_corr_cols
top_corr_columns = set(train_data.columns).intersection(set(top_corr_columns))
train_data_outliers = cleaning.remove_outliers_IQR_method(train_data, top_corr_columns)

TotalBsmtSF
Rows removed: 49

GarageCars
Rows removed: 5

GrLivArea
Rows removed: 7

GarageArea
Rows removed: 18

YearRemodAdd
Rows removed: 0

SalePrice
Rows removed: 17

1stFlrSF
Rows removed: 1

YearBuilt
Rows removed: 5

TotRmsAbvGrd
Rows removed: 16

FullBath
Rows removed: 0

GarageYrBlt
Rows removed: 1

OverallQual
Rows removed: 0


Rows removed in total: 119



In [19]:
# Run pipeline
grid_results, best_score = evaluate_feature_sets(train_data_outliers, 
                                                 reg=reg, scorer=scorer, cv=cv)

# Print best score
print(best_score)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.3s finished


0.12656302335354497


**Result:** Result on data with removed outliers leads to a better score.

In [20]:
# Check params
display(pd.DataFrame(grid_results).nsmallest(1, 'mean_test_score'))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessor__num__imputer_n__strategy,param_preprocessor__num__scaler,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
3,1.172863,0.108654,0.053246,0.022438,median,"StandardScaler(copy=True, with_mean=True, with...",{'preprocessor__num__imputer_n__strategy': 'me...,0.016128,0.015421,0.016506,0.016018,0.00045,4


#### Full train set with outliers removed and multi-correlation columns removed

In [21]:
# Remove Outliers for remaining top_corr_cols in the reduced data set
# GarageYrBlt was a the top_corr_features that was dropped above
cols_multi = set(train_data_outliers.columns).intersection(set(['1stFloor', 'GarageArea', 'FirstFlSF']))
train_data_multi = cleaning.delete_columns(train_data_outliers,  cols_multi)

assert train_data_multi.shape[1] == train_data_outliers.shape[1] - len(cols_multi)

'GarageArea successfully deleted'

In [22]:
# Run pipeline
grid_results, best_score = evaluate_feature_sets(train_data_multi, 
                                                 reg=reg, scorer=scorer, cv=cv)

# Print best score
print(best_score)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    3.9s finished


0.12658308430203355


**Result:** Result on data with removed multi_col(s) is slightly worse.

### Final Tuning & Evaluation

The full train_set with mean imputation, scaling and outlier removal scored the best results.

In [23]:
X, y = train_data_outliers.drop(['SalePrice'], axis = 1), train_data_outliers['SalePrice']
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [24]:
# Assemble pipeline (define function)
def build_final_pipe(X_train, y_train, reg):
    """Build a pipeline for preprocessing and modelling.
    
    ARGUMENTS:
        X_train: training features (df or array)
        y_train: training labels (df or array)
        reg: classifier (sk-learn model object)
        
    RETURNS:
        full_pipe: pipeline object
    """
    
    # Define cat and num feature columns
    categorical_features = X_train.select_dtypes(include=['category']).columns
    numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
    assert len(categorical_features) + len(numeric_features) == X_train.shape[1]
    
    # level 1 - two separate pipes for cat and num features
    numeric_transformer = Pipeline(steps=[
        ('imputer_n', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
            ])

    categorical_transformer = Pipeline(steps=[
        ('imputer_c', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ohe', OneHotEncoder(handle_unknown='ignore')),
            ])

    # level 2 - wrap the two level 1 pipes into a ColumnTransformer
    preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features),
                         ])

    # level 3 - pipe it with a classifier
    full_pipe = Pipeline(steps=[
                       ('preprocessor', preprocessor),
                       ('reg', reg),
                               ]) 
    
    return full_pipe

In [25]:
elastic_net_final = ElasticNetCV(l1_ratio=.9, 
                                 eps=1e-3, n_alphas=100, fit_intercept=True, 
                                 normalize=True, precompute='auto', max_iter=3000, 
                                 tol=0.0001, cv=6, copy_X=True, verbose=0, n_jobs=-1, 
                                 positive=False, random_state=0)

full_pipe_final = build_final_pipe(X_train, y_train, elastic_net_final) # cv on parameters
full_pipe_final.fit(X_train, y_train)

y_pred = full_pipe_final.predict(X_test)

In [26]:
print('Test r2 score: ', r2_score(y_test, y_pred))
test_mse = mean_squared_error(y_pred, y_test)
test_rmse = np.sqrt(test_mse)
print('Test RMSE: %.4f' % test_rmse)

Test r2 score:  0.9363510677997943
Test RMSE: 0.0887


In [28]:
# Save final model and data
joblib.dump(full_pipe_final, 'models/elastic_net_final.pkl')
train_data_outliers.to_csv('data/interim/train_opti_EN.csv', index=False)

---

## Apppendix: Experiment with preprocessing pipe only 

In [29]:
X, y = train_data_outliers.drop(['SalePrice'], axis = 1), train_data_outliers['SalePrice']

In [30]:
# Assemble pipeline (define function)
def build_preprocessing_pipe(X):
    """Build a pipeline for preprocessing and modelling.
    
    ARGUMENTS:
        X_train: training features (df or array)
        
    RETURNS:
        preprocessor: pipeline object
    """
    
    # Define cat and num feature columns
    categorical_features = X_train.select_dtypes(include=['category']).columns
    numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
    assert len(categorical_features) + len(numeric_features) == X_train.shape[1]
    
    # level 1 - two separate pipes for cat and num features
    numeric_transformer = Pipeline(steps=[
        ('imputer_n', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
            ])

    categorical_transformer = Pipeline(steps=[
        ('imputer_c', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ohe', OneHotEncoder(handle_unknown='ignore')),
            ])

    # level 2 - wrap the two level 1 pipes into a ColumnTransformer
    preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features),
                         ])
    
    return preprocessor

In [31]:
preprocessor = build_preprocessing_pipe(X)
X_tr = preprocessor.fit_transform(X)

---